In [11]:
'''
The code here:
    1. resamples CT and PET volumes to match a fixed size 
    2. isolates the GTV segmentation in RTSS files
    3. resamples labelmaps which were converted from the RTSS files using the BatchStructureSetConversion module in 3D slicer
    4. converts the CT/PET volumes and labelmaps into numpy arrays
'''

!pip install SimpleITK
!pip install pydicom

import pydicom
import SimpleITK as sitk
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import gzip
import shutil


In [2]:
# Trawl through files system for RTSS and volume files

def get_processed_dir(path):
    return path.replace('Head-Neck-PET-CT', 'Head-Neck-PET-CT-Processed')

def get_unprocessed_dir(path):
    return path.replace('Head-Neck-PET-CT-Processed', 'Head-Neck-PET-CT')

ct_structs = []
pet_structs = []
ct_vols = set()
pet_vols = set()

for root, dirs, files in os.walk('/home/jzhe0882/datasets/Head-Neck-PET-CT'):
    for name in files:
        file_path = os.path.join(root, name)
        
        relevant_dir = True #whether the directory contains a file to be processed
        if 'RTstructCTsim-CTPET-CT' in root:
            ct_structs.append(file_path) # CT RTSS file
            
        elif 'RTstructCTsim-PETPET-CT' in root:
            pet_structs.append(file_path) # PET RTSS file
            
        elif os.path.getsize(file_path) < 200000: 
            pet_vols.add(root) # Folder containing series of PET images

        elif os.path.getsize(file_path) < 1000000: 
            ct_vols.add(root) # Folder containing series of CT images
            
        else: #otherwise it is an RTDose file
            relevant_dir = False
            
        #create the relevant directories if they don't exist
        processed_dir = get_processed_dir(root)
        if relevant_dir and not os.path.isdir(processed_dir):
            os.makedirs(processed_dir) 
            
#all should be equal
print('num ct structs:', len(ct_structs))
print('num pet structs:', len(pet_structs))
print('num ct:', len(ct_vols))
print('num pet:', len(pet_vols))

num ct structs: 194
num pet structs: 194
num ct: 194
num pet: 194


In [3]:
# Associate each CT/PET volume with their RTSS file

vol2rtss = {} # associates each CT/PET volume directory with its paired RTSS' file path

for pet_path in pet_vols:    
    base_path = os.path.abspath(os.path.join(pet_path, '..'))
    for struct_path in pet_structs:
        if base_path in struct_path:
            vol2rtss[pet_path] = struct_path
            break
    else:
        print('RTSS not found for', root)
        
for ct_path in ct_vols:    
    base_path = os.path.abspath(os.path.join(ct_path, '..'))
    for struct_path in ct_structs:
        if base_path in struct_path:
            vol2rtss[ct_path] = struct_path
            break
    else:
        print('RTSS not found for', root)
        
print('vol2rtss size:', len(vol2rtss))

vol2rtss size: 388


In [5]:
# Remove all segmentations except for those corresponding to GTV's
# Associate the segmentations with the newly resampled CT/PET scans

#reference: https://dicom.innolitics.com/ciods/rt-structure-set/roi-contour/30060039/30060084

#check for potential gtv tag names if they aren't contained in gtv_names
missing_types = set()

#these are obtained from the names of gtv contours in https://wiki.cancerimagingarchive.net/display/Public/Head-Neck-PET-CT#5395a20070db41d49745c83e3312ff86
gtv_names = {'GTV', 'GTV1', 'GTV-P', 'GTV 67', 'gtv pet', 'GTV-BOT', 'GTV 67.5', 'GTV 67.5Gy', 'GTV primary_70GY',
           'GTV_67.5gy', 'GTV_70Gy', 'GTV_69Gy', 'GTVt', 'GTV primaire', 'GTV p', 'GTV Primaire 70', 'GTV T irm',
           'GTV 70', 'GTV et GG', 'GTV n', 'GTV_70gy', 'GTV_70GY', 'GTV_67.5', 'GTV-p', 'GTV_67.5Gy', 'GTV_67.5GY',
            'GTV_T_67.5gy'}

def configure_rtss(file, slice_sop_uids, patient_id):
    ds = pydicom.read_file(file)
    
    #there are cases where the patient id/name of the RTSS files are mismatched, so we use the id from the PET/CT volume metadata
    #patient_id = patient_name in this dataset
    ds.PatientID = patient_id
    ds.PatientName = patient_id
    
    gtv_num = None
    
    #print(dir(ds))
    #print(file)
    #print(ds.ReferencedFrameOfReferenceSequence)
    
    #Assign new SOP UID's for each of the slices
    for referenced_frame_of_reference in ds.ReferencedFrameOfReferenceSequence:       
        for RT_referenced_study in referenced_frame_of_reference.RTReferencedStudySequence:       
            for RT_referenced_series in RT_referenced_study.RTReferencedSeriesSequence:
                
                contour_image_sequence = RT_referenced_series.ContourImageSequence
                sop_class = contour_image_sequence[0].ReferencedSOPClassUID
                contour_image_sequence.clear()
                                
                for sop_uid in slice_sop_uids:
                    element = pydicom.Dataset()
                    element.ReferencedSOPClassUID = sop_class
                    element.ReferencedSOPInstanceUID = sop_uid
                    contour_image_sequence.append(element)
        break
    
    #look for the id number for the gross tumour volume tag
    for contour_type in ds.StructureSetROISequence:
        if contour_type.ROIName in gtv_names:
            gtv_num = contour_type.ROINumber
            break
     
    #shouldn't happen
    if gtv_num is None:
        
        for contour_type in ds.StructureSetROISequence:
            missing_types.add(contour_type.ROIName)
        
        print('GTV not found for', file)
        
    #remove all segmentations except the GTV segmentation
    else:       
        gtv_contours = []
        
        for contour in ds.ROIContourSequence:
            if contour.ReferencedROINumber == gtv_num:
                gtv_contours.append(contour)
                
        ds.ROIContourSequence.clear()
                
        for contour in gtv_contours:
            ds.ROIContourSequence.append(contour)
            
        ds.save_as(get_processed_dir(file))
        
#should be empty
print(missing_types)

set()


In [6]:
# Resample the CT and PET volumes

#Read-writing DICOM files is a delicate operation: https://simpleitk.readthedocs.io/en/master/Examples/DicomSeriesReadModifyWrite/Documentation.html

def resample_volume(path, ct=True):    
    target_ct_size = (128,128,96)
    target_pet_size = (128,128,96)
    
    reader = sitk.ImageSeriesReader()
    files = reader.GetGDCMSeriesFileNames(path)
    reader.SetFileNames(files)
    reader.MetaDataDictionaryArrayUpdateOn() #load tags
    reader.LoadPrivateTagsOn() #load private tags (these aren't loaded by default)
    volume = reader.Execute()
    
    #keep everything the same except for the spacing and size
    #'size' here refers to number of slices along each dimension
    #hence, total volume = spacing * size
    spacing = volume.GetSpacing()
    size = volume.GetSize()
    
    #In Python, variables introduced in if-else blocks preserve scope
    #Wtf
    if ct:
        new_size = target_ct_size
        new_spacing = ((size[0] / target_ct_size[0]) * spacing[0], 
                       (size[1] / target_ct_size[1]) * spacing[1], 
                       (size[2] / target_ct_size[2]) * spacing[2])
    else:
        new_size = target_pet_size
        new_spacing = ((size[0] / target_pet_size[0]) * spacing[0], 
                       (size[1] / target_pet_size[1]) * spacing[1], 
                       (size[2] / target_pet_size[2]) * spacing[2])
    
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetReferenceImage(volume)
    resampler.SetSize(new_size)
    resampler.SetOutputSpacing(new_spacing)
    
    new_volume = resampler.Execute(volume)
        
    writer = sitk.ImageFileWriter()  
    
    #by default, the image writer does not preserve meta-data from original DICOM series; we copy the following tags across
    writer.KeepOriginalImageUIDOn() 
    tags_to_copy = ['0002|0002', # Media Storage SOP Class UID
                '0008|0016', # SOP Class UID
                '0008|0022', # Acquisition Date
                '0008|0023', # Content Date
                '0008|0032', # Acquisition Time
                '0008|0033', # Content Time
                '0020|0012', # Acquisition Number
                "0010|0020", # Patient ID
                "0010|0030", # Patient Birth Date
                "0020|000d", # Study Instance UID, for machine consumption
                "0020|0010", # Study ID, for human consumption
                "0008|0020", # Study Date
                "0008|0030", # Study Time
                "0008|0050", # Accession Number
                "0008|0060", # Modality
                '0028|1050', # Window Centre
                '0028|1051' # Window Width    
                ]
        
    modification_time = time.strftime("%H%M%S")
    modification_date = time.strftime("%Y%m%d")
        
    tag_values = [(k, reader.GetMetaData(0,k)) for k in tags_to_copy if reader.HasMetaDataKey(0,k)] + \
                 [("0010|0010", reader.GetMetaData(0,"0010|0020")), # Patient ID is assigned to Patient Name 
                  ("0008|0031",modification_time), # Series Time
                  ("0008|0021",modification_date), # Series Date
                  ("0008|0008","DERIVED\\SECONDARY"), # Image Type
                  ('0018|0050', str(new_spacing[2])), # Slice Thickness
                  ('0028|0030', '{}/{}'.format(new_spacing[0], new_spacing[1])), # Pixel Spacing
                  ("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time), # Series Instance UID
                  ("0008|103e", reader.GetMetaData(0,"0008|103e") + " Processed-SimpleITK")] # Series Description
    
    #Some volumes need to cast from float to int before writing to the file system
    #Otherwise this error is thrown: A Floating point buffer was passed but the stored pixel type was not specified.
    castFilter = sitk.CastImageFilter()
    castFilter.SetOutputPixelType(sitk.sitkInt32)
    new_volume = castFilter.Execute(new_volume)
    
    #Write DICOM series while assigning metadata values
    save_directory = get_processed_dir(path)
    sop_uids = []
    for i in range(new_size[2]):
        image_slice = new_volume[:,:,i]
                
        #tags shared across all slices
        for tag, value in tag_values:
            image_slice.SetMetaData(tag, value)
        
        #slice-specific tags
        image_slice.SetMetaData("0008|0012", time.strftime("%Y%m%d")) # Instance Creation Date
        image_slice.SetMetaData("0008|0013", time.strftime("%H%M%S")) # Instance Creation Time
        image_slice.SetMetaData("0020|0032", '\\'.join(map(str,new_volume.TransformIndexToPhysicalPoint((0,0,i))))) # Image Position (Patient)
        image_slice.SetMetaData("0020|0013", str(i)) # Instance Number
        
        file_path = os.path.join(save_directory, '{0:06d}.dcm'.format(i))
        writer.SetFileName(file_path)
        writer.Execute(image_slice)
        
        #The SITK library can't read hierarchical data; we use pydicom to assign Referenced SOP Instance UID values
        #This is needed so that 3D Slicer recognises that this CT/PET scan is the source volume for RTSS segmentations
        source_volume = pydicom.read_file(files[0]) #the source volume can be any image slice as the following attributes are shared across all slices
        ds = pydicom.read_file(file_path)
        ds.SeriesInstanceUID = source_volume.SeriesInstanceUID
        ds.FrameOfReferenceUID = source_volume.FrameOfReferenceUID;
        if 'ReferencedStudySequence' in source_volume: #this field is sometimes omitted in some volumes
            ds.ReferencedStudySequence = source_volume.ReferencedStudySequence

        sop_uids.append(ds.SOPInstanceUID)
        ds.save_as(file_path)
    
    patient_id = reader.GetMetaData(0,"0010|0020")
    configure_rtss(vol2rtss[path], sop_uids, patient_id)
    
    #print('completed {}'.format(save_directory))
    #print('Size:', volume.GetSize(), '->', new_volume.GetSize())
    #print('Spacing:', volume.GetSpacing(), '->', new_volume.GetSpacing())
        
        
#resample_volume(path='/home/jzhe0882/datasets/Head-Neck-PET-CT/HN-CHUM-007/08-27-1885-CT pour planification -ONCO-09347/837620-LOR-RAMLA-92958', ct=False)

resampling_progress = 0
for ct_path in ct_vols:
    resample_volume(ct_path, ct=True)
    
    resampling_progress += 1
    if resampling_progress % 20 == 0:
        print('resampled', resampling_progress)
        
for pet_path in pet_vols:
    resample_volume(pet_path, ct=False)
    
    resampling_progress += 1
    if resampling_progress % 20 == 0:
        print('resampled', resampling_progress)
        
print('resampled', resampling_progress)

resampled 20
resampled 40
resampled 60
resampled 80
resampled 100
resampled 120
resampled 140
resampled 160
resampled 180
resampled 200
resampled 220
resampled 240
resampled 260
resampled 280
resampled 300
resampled 320
resampled 340
resampled 360
resampled 380
resampled 388


In [7]:
# Scan file system for masks generated by 3D Slicer and associate them with corresponding CT/PET volumes

def get_processed_mask_dir(path):
    return path.replace('LabelMaps', 'LabelMaps-Processed')

mask2pet = {} #maps masks to resampled pet images' directories
mask2ct = {} #maps masks to resampled ct images' directories

for root, dirs, files in os.walk('/home/jzhe0882/datasets/LabelMaps'):
    patient = os.path.basename(root) #e.g. HN-CHUM-001
    
    #choose the labelmap with the largest filesize to be the mask for both the CT and PET volume
    mask = None
    max_size = 0
    for name in files:
        file_path = os.path.join(root, name)
        file_size = os.path.getsize(file_path)
        if file_size > max_size:
            max_size = file_size
            mask = file_path
            
        #create the directory to save the resampled label maps 
        processed_dir = get_processed_mask_dir(root)
        if not os.path.isdir(processed_dir):
            os.makedirs(processed_dir) 
            
    #map the ct and pet volume to the mask
    if mask is not None:
        for volume in ct_vols:
            if patient in volume:
                mask2ct[mask] = get_processed_dir(volume)
                break
                
        for volume in pet_vols:
            if patient in volume:
                mask2pet[mask] = get_processed_dir(volume)
                break
    else:
        print('mask not found in', root)
                
#should be equal
print('mask2pet:', len(mask2pet))
print('mask2ct:', len(mask2ct))

mask not found in /home/jzhe0882/datasets/LabelMaps
mask2pet: 194
mask2ct: 194


In [8]:
def resample_mask(mask_dir, reference_dir):
    
    #load mask volume
    mask_reader = sitk.ImageFileReader()
    mask_reader.SetFileName(mask_dir)
    mask_volume = mask_reader.Execute()
    
    #load ct/pet volume
    reference_reader = sitk.ImageSeriesReader()
    reference_files = reference_reader.GetGDCMSeriesFileNames(reference_dir)
    reference_reader.SetFileNames(reference_files)
    #reference_reader.MetaDataDictionaryArrayUpdateOn() #load tags
    #reference_reader.LoadPrivateTagsOn() #load private tags (these aren't loaded by default)
    reference_volume = reference_reader.Execute()
    
    #resample mask volume to match reference image parameters
    resampler = sitk.ResampleImageFilter()
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    resampler.SetReferenceImage(reference_volume)
    new_mask = resampler.Execute(mask_volume)
    
    '''castFilter = sitk.CastImageFilter()
    castFilter.SetOutputPixelType(sitk.sitkInt32)
    new_mask = castFilter.Execute(new_mask)'''
    
    writer = sitk.ImageFileWriter()  
    writer.KeepOriginalImageUIDOn()
    writer.SetFileName(get_processed_mask_dir(mask_dir))
    writer.Execute(new_mask)
    

In [9]:
# Resample label maps that were converted from the RTSS files using 3D slicer

resampling_progress = 0

#use the PET images for mask generation, as it has lower resolution
for mask, volume in mask2pet.items():
    resample_mask(mask, volume)
    
    resampling_progress += 1
    if resampling_progress % 20 == 0:
        print('resampled', resampling_progress)
        
print('resampled', resampling_progress)
#resample_mask('/home/jzhe0882/datasets/LabelMaps/HN-CHUM-007/1  RTSTRUCT  RTstruct_1_GTV.nrrd',
#             '/home/jzhe0882/datasets/Head-Neck-PET-CT/HN-CHUM-007/08-27-1885-CT pour planification -ONCO-09347/837620-LOR-RAMLA-92958')

resampled 20
resampled 40
resampled 60
resampled 80
resampled 100
resampled 120
resampled 140
resampled 160
resampled 180
resampled 194


In [14]:
# Convert PET/CT volumes and segmentation masks to numpy arrays

def convert(source_path, target_path, source_type='volume', target_type='numpy'):
    
    reader = None
    if source_type == 'volume':
        reader = sitk.ImageSeriesReader()
        file_names = reader.GetGDCMSeriesFileNames(source_path)
        reader.SetFileNames(file_names)
    elif source_type == 'single_image':
        reader = sitk.ImageFileReader()
        reader.SetFileName(source_path)
    else:
        print('incorrect source type')
        return
        
    volume = reader.Execute()
    
    if target_type == 'numpy':
        target_path = '{}.npy'.format(target_path)
        array = sitk.GetArrayFromImage(volume)
        np.save(target_path, np.transpose(array))
    elif target_type == 'nifti':
        target_path = '{}.nii'.format(target_path)
        writer = sitk.ImageFileWriter()
        writer.SetFileName(target_path)
        writer.Execute(volume)
        
        with open(target_path, 'rb') as f_in: #compress into .gz file as 3D-DSN code only accepts .nii.gz files :/
            with gzip.open('{}.gz'.format(target_path), 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
    
    
index = 0
for mask, pet in mask2pet.items():
    ct = mask2ct[mask]
    mask = get_processed_mask_dir(mask)
    
    #e.g. HN-CHUM-001
    patient = os.path.basename(os.path.abspath(os.path.join(ct, '../..')))
    
    #numpy conversion
    numpy_ct_path = os.path.join('/home/jzhe0882/numpydata/CT', patient)
    numpy_pet_path = os.path.join('/home/jzhe0882/numpydata/PET', patient)
    numpy_mask_path = os.path.join('/home/jzhe0882/numpydata/Mask', patient)
    
    convert(ct, numpy_ct_path, source_type='volume', target_type='numpy')
    convert(pet, numpy_pet_path, source_type='volume', target_type='numpy')
    convert(mask, numpy_mask_path, source_type='single_image', target_type='numpy')
    
    #also convert files to a NIFTI format for other frameworks; VERY SLOW BLYAT
    nifti_ct_path = os.path.join('/home/jzhe0882/datasets/Nifti', 'training_axial_crop_pat{}'.format(index))
    #nifti_pet_path = os.path.join('/home/jzhe0882/datasets/Nifti', 'training_axial_crop_pat{}'.format(index))
    nifti_mask_path = os.path.join('/home/jzhe0882/datasets/Nifti', 'training_axial_crop_pat{}-label'.format(index))
    
    convert(ct, nifti_ct_path, source_type='volume', target_type='nifti')
    #convert(pet, nifti_pet_path, source_type='volume', target_type='nifti')
    convert(mask, nifti_mask_path, source_type='single_image', target_type='nifti')
    
    index += 1
    if index % 20 == 0:
        print('converted', index)
        
print('converted', index)

converted 20
converted 40
converted 60
converted 80
converted 100
converted 120
converted 140
converted 160
converted 180
converted 194
